In [232]:
import os
import json
import pandas as pd
import numpy as np
if os.getcwd() != '/home/jupyter/crisp':
    os.chdir("..")
from utils.gcp_helpers import get_dataframe_from_bucket
%pwd

'/home/jupyter/crisp'

In [233]:
human_data = get_dataframe_from_bucket("Radiation/human/combined/human.combined.expression.071321.tsv","ah21_data", "ah21_data", sep="\t")
mouse_data = get_dataframe_from_bucket("Radiation/mouse/combined/mouse.combined.expression.071121.tsv","ah21_data", "ah21_data", sep="\t")
human_mouse_gene_map = pd.read_csv('data/mouse_human_mapping.txt', sep='\t')

db_mouse_genes = set(human_mouse_gene_map['Mouse gene name'].str.lower())
db_human_genes = set(human_mouse_gene_map['Gene name'].str.lower())

experiment_human_genes = set(human_data.GeneName.str.lower())
experiment_mouse_genes = set(mouse_data.GeneName.str.lower())
human_subset = experiment_human_genes.intersection(db_human_genes)
mouse_subset = experiment_mouse_genes.intersection(db_mouse_genes)

In [234]:
human_to_mouse_dic = dict(zip(human_mouse_gene_map['Gene name'].str.lower(),human_mouse_gene_map['Mouse gene name'].str.lower()))
mouse_to_human_dic = dict(zip(human_mouse_gene_map['Mouse gene name'].str.lower(),human_mouse_gene_map['Gene name'].str.lower()))

In [235]:
humanised_mouse_genes = set([mouse_to_human_dic[x] for x in list(mouse_subset)])

In [236]:
human_genes_to_use = list(humanised_mouse_genes.intersection(human_subset))
mouse_genes_to_use = ([human_to_mouse_dic[x] for x in human_genes_to_use])

In [237]:
h_data = get_dataframe_from_bucket("Radiation/human/combined/human.combined.expression.071321.tsv","ah21_data", "ah21_data", sep="\t")
h_data.GeneName = h_data.GeneName.str.lower()
h_data = h_data.T
h_data.columns = h_data.iloc[0]
h_data = h_data.iloc[1:].astype(float)
h_data = h_data[human_genes_to_use]

h_metadata = get_dataframe_from_bucket("Radiation/human/combined/human.metadata.combined.071321.csv","ah21_data", "ah21_data", sep=",")
h_metadata['environment'] = h_metadata.dataset + h_metadata.gender + h_metadata.organism.str[:3]
h_merged_data = pd.merge(h_metadata, h_data, left_on="sample", right_index=True)
h_merged_data["irradiated"] = [1. if x == "Yes" else 0. for x in  h_merged_data.irradiated.values]

In [238]:
m_data = get_dataframe_from_bucket("Radiation/mouse/combined/mouse.combined.expression.071121.tsv","ah21_data", "ah21_data", sep="\t")
m_data.GeneName = m_data.GeneName.str.lower().map(mouse_to_human_dic)
m_data = m_data.drop_duplicates('GeneName')
m_data = m_data.T
m_data.columns = m_data.iloc[0]
m_data = m_data.iloc[1:].astype(float)
m_data = m_data[human_genes_to_use]

m_metadata = get_dataframe_from_bucket("Radiation/mouse/combined/mouse.metadata.combined.071121.csv","ah21_data", "ah21_data", sep=",")
m_metadata['environment'] = m_metadata.dataset + m_metadata.gender + m_metadata.organism.str[:3]
m_merged_data = pd.merge(m_metadata, m_data, left_on="sample", right_index=True)
m_merged_data["irradiated"] = [1. if x == "Yes" else 0. for x in  m_merged_data.irradiated.values]
m_merged_data = m_merged_data[m_merged_data.radiation != 'Low-dose-rate_Xray']

In [239]:
h_m_data = h_merged_data.append(m_merged_data)

In [240]:
h_m_data

,sample,organism,source,gender,age_years,timepoint_hr,dose_Gy,radiation,dataset,irradiated,...,pnrc1,scarb2,gtf2h5,gbe1,fut1,ptch2,mrgpre,ube2e3,arhgap10,strain
0,GSM226029,Homo sapiens,Peripheral Blood,female,23.0,24,0.0,Control,GLDS152,0.0,...,1.766562,-0.202445,-0.209839,0.073109,-0.291794,-0.256510,-0.275536,0.017463,-0.251963,NaN
1,GSM226014,Homo sapiens,Peripheral Blood,male,47.0,24,0.0,Control,GLDS152,0.0,...,2.210526,-0.232310,-0.210733,0.012674,-0.270642,-0.250864,-0.260006,0.024065,-0.236310,NaN
2,GSM225984,Homo sapiens,Peripheral Blood,male,39.0,6,0.0,Control,GLDS152,0.0,...,1.666417,-0.243300,-0.198792,-0.067264,-0.269195,-0.251835,-0.261581,0.056643,-0.244759,NaN
3,GSM226024,Homo sapiens,Peripheral Blood,female,46.0,24,0.0,Control,GLDS152,0.0,...,1.679884,-0.241021,-0.223672,0.023216,-0.277267,-0.264870,-0.277727,0.034405,-0.252363,NaN
4,GSM225989,Homo sapiens,Peripheral Blood,female,20.0,6,0.0,Control,GLDS152,0.0,...,1.802661,-0.241177,-0.196736,-0.103085,-0.274432,-0.260511,-0.274162,0.085708,-0.256447,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
309,GSM1530198,Mus musculus,NaN,male,NaN,24,4.0,High-dose-rate_Xray,GSE62623,1.0,...,-0.211977,-0.228946,3.854475,-0.220380,-0.230628,2.513553,-0.230093,-0.187481,-0.229182,C57BL/6
310,GSM1530199,Mus musculus,NaN,male,NaN,24,4.0,High-dose-rate_Xray,GSE62623,1.0,...,-0.151436,-0.160579,0.395899,-0.152114,-0.169940,0.244645,-0.164498,-0.035327,-0.158337,C57BL/6
311,GSM1530200,Mus musculus,NaN,male,NaN,24,4.0,High-dose-rate_Xray,GSE62623,1.0,...,-0.162801,-0.146230,0.352651,-0.187800,-0.264375,0.494371,-0.256079,0.056382,-0.257785,C57BL/6
312,GSM1530201,Mus musculus,NaN,male,NaN,24,4.0,High-dose-rate_Xray,GSE62623,1.0,...,-0.150376,-0.151990,0.486711,-0.153037,-0.170671,0.231586,-0.164833,-0.040300,-0.159622,C57BL/6


In [228]:
h_m_data.environment.value_counts()

GSE124612maleMus    248
GLDS157femaleHom     40
GLDS157maleHom       35
GSE62623maleMus      30
GLDS152maleHom       25
GLDS152femaleHom     25
GLDS156maleMus       18
Name: environment, dtype: int64

In [227]:
len(h_m_data[h_m_data.organism == 'Homo sapiens'])

125

In [230]:
h_m_data.to_pickle('data/combined_mouse_human_data.pkl')